In [4]:
import netCDF4 as nc

In [5]:
root = nc.Dataset("test.nc", "r")

In [38]:
root["base_time"][:]

array(1387756800, dtype=int32)

In [43]:
root["time_offset"]

<class 'netCDF4._netCDF4.Variable'>
float64 time_offset(time)
    long_name: Time offset from base_time
    units: seconds since 2013-12-23 00:00:00 0:00
    ancillary_variables: base_time
unlimited dimensions: time
current shape = (1440,)
filling off

In [44]:
root

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_CLASSIC data model, file format NETCDF3):
    Conventions: ARM-1.1
    command_line: radflux1long_vap -s mao -f M1 -b 20131220 -e 20151201 -n radflux1longc2 require_goodday 0 -D 2
    process_version: vap-radflux1long-2.3-0.el6
    dod_version: radflux1long-c2-1.4
    site_id: mao
    facility_id: M1
    platform_id: radflux1long
    location_description: Green Ocean Amazon (GOAMAZON), Manacapuru, Amazonia, Brazil
    data_level: c2
    rh_factor_flag: 1
    rh_factor_flag_comment: 0 = no_coef_applied 1 = calculate_coef 2 = use_std_coef 3 = use_coef_from_prev_run
    fitmode: 0
    fitmode_comment: 01 = daily_fit 00 =  1_fit
    input_datastreams: maoqcrad1longM1.c2 : 0.5 : 20131220.000000-20151201.000000
    datastream: maoradflux1longM1.c2
    doi: 10.5439/1157585
    history: created by user gaustad on machine amber at 2016-03-18 19:55:19, using vap-radflux1long-2.3-0.el6
    dimensions(sizes): time(1440), bound(2)
    variables

In [53]:
root["base_time"]

<class 'netCDF4._netCDF4.Variable'>
int32 base_time()
    string: 2013-12-23 00:00:00 0:00
    long_name: Base time in Epoch
    units: seconds since 1970-1-1 0:00:00 0:00
    ancillary_variables: time_offset
unlimited dimensions: 
current shape = ()
filling off